# Dune Migration

_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_

In this notebook, we're going to introduce temporal datasets in GRASS to see how Jockeys Ridge has moved over the last two decades. We'll create a animation showing the migration.

***

## 1. Import Python Packages and Start GRASS Session

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("nags_head/PERMANENT");

Before we get started, we set and check the computational region.

In [ ]:
!g.region region=jockeys_ridge -p

***

## 2. Getting Started with GRASS Temporal Tools


To better handle the long time series of maps, GRASS has a temporal datasets which serve as containers for the individual layers in the time series. We can further manipulate them all together in the temporal dataset instead of the individual maps. For example, we can easily aggregate data at different time intervals, compute univariate statics, or set a univeral color scheme.

These temporal dataset can contain rasters (space-time raster dataset or _strds_), 3D rasters (_str3ds_) or vectors (_stvds_). Visit [the temporal data processing manual](https://grass.osgeo.org/grass83/manuals/temporalintro.html) for more info.

First, we create empty datasets of type strds (space-time raster dataset).

In [ ]:
gs.run_command('t.create', output='JockeysRidge', type='strds',
                temporaltype='relative', title="Jockeys Ridge Elevation Series",
                description="from 1996 to 2020 with gaps")

After defining our temporal dataset, we can add and remove layers from it with `t.register` and `t.unregister`. 

In [ ]:
# Get list of rasters we'd like to add to our dataset
DEMs = gs.read_command("g.list", type="raster", pattern="JR_????", separator="comma").strip().split(",")
DEMs

In [ ]:
# Get the year of each DEM
years=[name[-4:] for name in DEMs]
years

In [ ]:
# Put DEM name and date together in table to register them
table = ""

for DEM, year in zip(DEMs, years):
    row = f"{DEM}|{year}\n"
    table += row

# Print formatted table
print(table)

# Write the formatted table to a text file
with open('./table.txt', 'w') as file:
    file.write(table)

In [ ]:
# Register rasters to dataset
gs.run_command("t.register", input="JockeysRidge", file="./table.txt", type="raster", unit="years")

Check what you have now in JockeysRidge dataset.

In [ ]:
print(gs.read_command("t.rast.list", input="JockeysRidge", columns="name,start_time"))

Set the same color table for all maps.

In [ ]:
gs.run_command("t.rast.colors", input="JockeysRidge", color="elevation")

Animate the time series.

In [ ]:
!r.mask raster=JR_2014

In [ ]:
animation = gj.TimeSeriesMap()
animation.add_raster_series("JockeysRidge", fill_gaps=True)
animation.d_legend(color="black", at=(12,72,0,3), fontsize=12)
animation.d_barscale(length=250)
# animation.show()

In [ ]:
animation.save(filename="migration.gif");

In [ ]:
from IPython.display import display, Image

display(Image(filename="migration.gif"))

**Try it Yourself!**
** Qn 1**

_The animation reveals that the datasets for the years 1996, 1997, 1998, 2000, 2004, 2005, 2006, and 2007 are incomplete. Modify the code above to adjust the DEM list to include only the years with complete datasets._

<details>
    <summary>👉 <b>Hint!!</b></summary>

```python
DEMs=['JR_2001','JR_2008','JR_2014','JR_2016','JR_2018','JR_2020']
``` 
</details>


**Hurricane Floyd was a very powerful Cape Verde hurricane which struck the Bahamas and the East Coast of the United States. It was the sixth named storm, fourth hurricane, and third major hurricane in the 1999 Atlantic hurricane season.**

![](img/Floyd_banner.png)

**We have lidar datasets that were taken before and after hurricane (JR_1999_0909, JR_1999_0918). Using the dataset provided, calculate the difference in landmass volume before and after the impact of Hurricane Floyd. Also, compute the net volume change**

<details>
    <summary>👉 <b>Hints!!!</b></summary>

```python
!r.mapcalc "elevation_difference = JR_1999_1104 - JR_1998"
``` 
    
```python
stats= gs.parse_command("r.univar", map='elevation_difference', flags='g')
stats['sum']
print(f"Net sand change is {stats ['sum']} cubic meter")
```
</details>


Calculate the DEM difference!!!

In [ ]:
!r.mapcalc "elevation_difference = JR_1999_1104 - JR_1998"

In [ ]:
!r.colors map=elevation_difference color=difference
example = gj.Map()
example.d_background(color="white")
example.d_rast(map="elevation_difference") # d.rast map=naip_2020.1
example.d_legend(raster="elevation_difference")
#example.d_barscale() # d.barscale
example.show()

Now, calculate the change in volume of sand.

In [ ]:
stats= gs.parse_command("r.univar", map='elevation_difference', flags='g')
stats['sum']
print(f"Net sand change is {stats ['sum']} cubic meter")